# Signal Validation: ggH → H → Za (NanoAODv15)

CMS-style validation plots for the locally produced NanoAODv15 signal sample.

**Signal process:** $gg \to H(125) \to Z(\to \ell^+\ell^-) + a(\to \text{tracks})$ at $\sqrt{s} = 13$ TeV

**Sample:** 83 events, $m_a = 1.0$ GeV, no pileup (chain validation run)

Uses `uproot` + `awkward` for columnar analysis and `mplhep` for CMS-style plotting.

In [ ]:
import uproot
import awkward as ak
import numpy as np
import vector
import hist
import matplotlib.pyplot as plt
import mplhep as hep

# Print versions for reproducibility
for mod in [uproot, ak, np, vector, hist, hep]:
    print(f"{mod.__name__:12s} {mod.__version__}")

vector.register_awkward()

: 

In [ ]:
# ── CMS plot style ──────────────────────────────────────────────────────────────
hep.style.use(hep.style.CMS)
plt.rcParams.update({"figure.figsize": (10, 8), "font.size": 16})

LUMI_TEXT = ""  # no luminosity for private MC
MASS_A = 1.0   # GeV — signal hypothesis

def cms_label(ax, label="Simulation Work in Progress"):
    """Add CMS label to axes."""
    hep.cms.label(label, data=False, ax=ax, loc=0, fontsize=18, com=13)

def save_and_show(fig, name=None):
    """Tight-layout, optionally save, then show."""
    fig.tight_layout()
    if name:
        fig.savefig(name, dpi=150, bbox_inches="tight")
    plt.show()

## Load Signal NanoAOD

Read the NanoAODv15 file from the full-chain local production run. Print event count and available collections.

In [ ]:
# ── Load NanoAOD ───────────────────────────────────────────────────────────────
NANO_FILE = "/eos/user/p/pgadow/www/share/RunIII2024Summer24NanoAODv15_ggH_HZa_mA1_0GeV.root"

tree = uproot.open(f"{NANO_FILE}:Events")
print(f"Events: {tree.num_entries}")

# Read all needed branches at once
branches_to_read = [
    # GenPart
    "nGenPart", "GenPart_pt", "GenPart_eta", "GenPart_phi", "GenPart_mass",
    "GenPart_pdgId", "GenPart_status", "GenPart_statusFlags", "GenPart_genPartIdxMother",
    # Muons
    "nMuon", "Muon_pt", "Muon_eta", "Muon_phi", "Muon_mass", "Muon_charge",
    "Muon_mediumId", "Muon_pfRelIso04_all", "Muon_dxy", "Muon_dz",
    # Electrons
    "nElectron", "Electron_pt", "Electron_eta", "Electron_phi", "Electron_mass",
    "Electron_charge", "Electron_cutBased", "Electron_pfRelIso03_all",
    "Electron_dxy", "Electron_dz",
    # Jets
    "nJet", "Jet_pt", "Jet_eta", "Jet_phi", "Jet_mass",
    "Jet_btagDeepFlavB",
    # MET (NanoAODv15 uses PuppiMET)
    "PuppiMET_pt", "PuppiMET_phi",
    # IsoTracks (for a reconstruction)
    "nIsoTrack", "IsoTrack_pt", "IsoTrack_eta", "IsoTrack_phi",
    "IsoTrack_charge", "IsoTrack_pdgId",
    "IsoTrack_pfRelIso03_all", "IsoTrack_fromPV",
    "IsoTrack_isHighPurityTrack", "IsoTrack_dxy", "IsoTrack_dz",
]

# Check for PFCands branches (available in NanoAODv15 with addPFCands customisation)
all_keys = tree.keys()
pf_keys = [k for k in all_keys if k.startswith("PFCands_")]
print(f"\nPFCands branches found: {len(pf_keys)}")
if pf_keys:
    print("  ", pf_keys[:15], "..." if len(pf_keys) > 15 else "")
    # Add PFCands branches
    pf_branches = [k for k in pf_keys if any(sub in k for sub in
        ["_pt", "_eta", "_phi", "_mass", "_charge", "_pdgId", "_dz", "_d0",
         "_pvAssocQuality", "_trkChi2", "_vertexChi2", "_fromPV"])]
    branches_to_read.extend(pf_branches)
    print(f"  Adding {len(pf_branches)} PFCands branches to read list")
else:
    # If PFCands not stored, look for alternative: FsrPhoton, LowPtElectron, SoftActivityJet
    alt_collections = ["FsrPhoton", "LowPtElectron", "SoftActivityJet", "SubJet"]
    for coll in alt_collections:
        coll_keys = [k for k in all_keys if k.startswith(f"{coll}_")]
        if coll_keys:
            print(f"  Alternative collection found: {coll} ({len(coll_keys)} branches)")

HAS_PFCANDS = len(pf_keys) > 0

# Read into awkward arrays
events = tree.arrays(branches_to_read, library="ak")
print(f"\nLoaded {len(events)} events with {len(branches_to_read)} branches")

## Generator-Level Kinematic Distributions

Extract the Higgs ($H$, PDG 25), Z boson (PDG 23), and pseudoscalar $a$ (PDG 36) from the `GenPart` collection using status flags. Plot their $p_T$, $\eta$, and mass distributions.

In [ ]:
# ── Extract gen-level particles ────────────────────────────────────────────────
genpart = ak.zip({
    "pt":     events.GenPart_pt,
    "eta":    events.GenPart_eta,
    "phi":    events.GenPart_phi,
    "mass":   events.GenPart_mass,
    "pdgId":  events.GenPart_pdgId,
    "status": events.GenPart_status,
    "statusFlags": events.GenPart_statusFlags,
    "motherIdx": events.GenPart_genPartIdxMother,
})

# isLastCopy flag = bit 13 of statusFlags
is_last_copy = (genpart.statusFlags & (1 << 13)) != 0

# Select gen-level particles (last copy)
gen_higgs = genpart[(abs(genpart.pdgId) == 25) & is_last_copy]
gen_Z     = genpart[(abs(genpart.pdgId) == 23) & is_last_copy]
gen_a     = genpart[(abs(genpart.pdgId) == 36) & is_last_copy]

# Also get gen-level leptons from Z decay (status 1 = stable)
gen_leptons = genpart[
    ((abs(genpart.pdgId) == 11) | (abs(genpart.pdgId) == 13)) &
    (genpart.status == 1) & is_last_copy
]

print(f"Gen Higgs per event: {ak.mean(ak.num(gen_higgs)):.1f}")
print(f"Gen Z per event:     {ak.mean(ak.num(gen_Z)):.1f}")
print(f"Gen a per event:     {ak.mean(ak.num(gen_a)):.1f}")
print(f"Gen leptons/event:   {ak.mean(ak.num(gen_leptons)):.1f}")

In [ ]:
# ── Gen-level plots: Higgs, Z, a ──────────────────────────────────────────────
fig, axes = plt.subplots(2, 3, figsize=(21, 13))

# Flatten to 1D numpy arrays for histogramming
h_pt  = ak.flatten(gen_higgs.pt).to_numpy()
z_pt  = ak.flatten(gen_Z.pt).to_numpy()
a_pt  = ak.flatten(gen_a.pt).to_numpy()
h_m   = ak.flatten(gen_higgs.mass).to_numpy()
z_m   = ak.flatten(gen_Z.mass).to_numpy()
a_m   = ak.flatten(gen_a.mass).to_numpy()
h_eta = ak.flatten(gen_higgs.eta).to_numpy()
z_eta = ak.flatten(gen_Z.eta).to_numpy()
a_eta = ak.flatten(gen_a.eta).to_numpy()

# Row 1: pT distributions
for ax, data, label, color, xmax in zip(
    axes[0],
    [h_pt, z_pt, a_pt],
    [r"$H(125)$", r"$Z$", r"$a$"],
    ["#e74c3c", "#3498db", "#2ecc71"],
    [300, 200, 200],
):
    h = hist.Hist(hist.axis.Regular(30, 0, xmax, label=r"$p_T$ [GeV]"))
    h.fill(data)
    hep.histplot(h, ax=ax, histtype="fill", alpha=0.5, color=color, edgecolor=color, label=label)
    ax.set_ylabel("Events")
    ax.legend(fontsize=16)
    cms_label(ax)

# Row 2: mass distributions
for ax, data, label, color, xrange, vline in zip(
    axes[1],
    [h_m, z_m, a_m],
    [r"$m_H$", r"$m_Z$", r"$m_a$"],
    ["#e74c3c", "#3498db", "#2ecc71"],
    [(120, 130), (0, 120), (0, 3)],
    [125.0, 91.2, MASS_A],
):
    h = hist.Hist(hist.axis.Regular(30, *xrange, label="Mass [GeV]"))
    h.fill(data)
    hep.histplot(h, ax=ax, histtype="fill", alpha=0.5, color=color, edgecolor=color, label=label)
    ax.axvline(vline, color="black", linestyle="--", linewidth=1.5, label=f"Nominal = {vline} GeV")
    ax.set_ylabel("Events")
    ax.legend(fontsize=14)
    cms_label(ax)

save_and_show(fig, "gen_kinematics.png")

In [ ]:
# ── Gen-level angular distributions ───────────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(21, 6))

for ax, data, label, color in zip(
    axes,
    [h_eta, z_eta, a_eta],
    [r"$H(125)$", r"$Z$", r"$a$"],
    ["#e74c3c", "#3498db", "#2ecc71"],
):
    h = hist.Hist(hist.axis.Regular(30, -6, 6, label=r"$\eta$"))
    h.fill(data)
    hep.histplot(h, ax=ax, histtype="fill", alpha=0.5, color=color, edgecolor=color, label=label)
    ax.set_ylabel("Events")
    ax.legend(fontsize=16)
    cms_label(ax)

save_and_show(fig, "gen_eta.png")

In [ ]:
# ── Gen-level ΔR(Z, a) and dilepton mass ─────────────────────────────────────
# Build Lorentz vectors for Z and a (per-event, take first candidate)
gen_Z_vec = vector.zip({
    "pt": gen_Z.pt[:, 0:1], "eta": gen_Z.eta[:, 0:1],
    "phi": gen_Z.phi[:, 0:1], "mass": gen_Z.mass[:, 0:1],
})
gen_a_vec = vector.zip({
    "pt": gen_a.pt[:, 0:1], "eta": gen_a.eta[:, 0:1],
    "phi": gen_a.phi[:, 0:1], "mass": gen_a.mass[:, 0:1],
})

gen_dR_Za = ak.flatten(gen_Z_vec.deltaR(gen_a_vec)).to_numpy()
gen_dphi_Za = ak.flatten(np.abs(gen_Z_vec.deltaphi(gen_a_vec))).to_numpy()

# Gen-level lepton pT and eta (use gen_leptons defined above)
gen_lep_pt  = ak.flatten(gen_leptons.pt).to_numpy()
gen_lep_eta = ak.flatten(gen_leptons.eta).to_numpy()

fig, axes = plt.subplots(2, 2, figsize=(14, 13))

# ΔR(Z, a)
h = hist.Hist(hist.axis.Regular(25, 0, 5, label=r"$\Delta R(Z, a)$"))
h.fill(gen_dR_Za)
hep.histplot(h, ax=axes[0, 0], histtype="fill", alpha=0.5, color="#9b59b6", edgecolor="#9b59b6")
axes[0, 0].set_ylabel("Events")
cms_label(axes[0, 0])

# Δφ(Z, a)
h = hist.Hist(hist.axis.Regular(25, 0, np.pi, label=r"$|\Delta\phi(Z, a)|$"))
h.fill(gen_dphi_Za)
hep.histplot(h, ax=axes[0, 1], histtype="fill", alpha=0.5, color="#e67e22", edgecolor="#e67e22")
axes[0, 1].set_ylabel("Events")
cms_label(axes[0, 1])

# Gen lepton pT
h = hist.Hist(hist.axis.Regular(30, 0, 100, label=r"Gen lepton $p_T$ [GeV]"))
h.fill(gen_lep_pt)
hep.histplot(h, ax=axes[1, 0], histtype="fill", alpha=0.5, color="#1abc9c", edgecolor="#1abc9c")
axes[1, 0].set_ylabel("Leptons / bin")
cms_label(axes[1, 0])

# Gen lepton eta
h = hist.Hist(hist.axis.Regular(30, -5, 5, label=r"Gen lepton $\eta$"))
h.fill(gen_lep_eta)
hep.histplot(h, ax=axes[1, 1], histtype="fill", alpha=0.5, color="#1abc9c", edgecolor="#1abc9c")
axes[1, 1].set_ylabel("Leptons / bin")
cms_label(axes[1, 1])

save_and_show(fig, "gen_deltaR_leptons.png")

In [ ]:
# ── Gen-level dilepton invariant mass (mll) ───────────────────────────────────
# Build Lorentz vectors for gen leptons and compute all OSSF pair masses
gen_lep_vec = vector.zip({
    "pt": gen_leptons.pt, "eta": gen_leptons.eta,
    "phi": gen_leptons.phi, "mass": gen_leptons.mass,
})
gen_lep_pdgId = gen_leptons.pdgId

# All pairs
gen_pairs = ak.argcombinations(gen_lep_vec, 2, axis=1)
gi0, gi1 = ak.unzip(gen_pairs)
gl0 = gen_lep_vec[gi0]
gl1 = gen_lep_vec[gi1]
gq0 = gen_lep_pdgId[gi0]
gq1 = gen_lep_pdgId[gi1]

# Opposite-sign same-flavour
ossf_mask = (gq0 + gq1 == 0)  # e.g. +11 + -11 = 0
gl0_ossf = gl0[ossf_mask]
gl1_ossf = gl1[ossf_mask]
gen_mll = (gl0_ossf + gl1_ossf).mass

gen_mll_flat = ak.flatten(gen_mll).to_numpy()

fig, ax = plt.subplots(figsize=(10, 8))
h = hist.Hist(hist.axis.Regular(50, 0, 150, label=r"$m_{\ell\ell}$ [GeV]"))
h.fill(gen_mll_flat)
hep.histplot(h, ax=ax, histtype="fill", alpha=0.5, color="#3498db", edgecolor="#3498db",
             label=f"Gen OSSF pairs (N={len(gen_mll_flat)})")
ax.axvline(91.2, color="black", linestyle="--", linewidth=1.5, label=r"$m_Z$ = 91.2 GeV")
ax.set_ylabel("Lepton pairs / bin")
ax.legend(fontsize=14)
cms_label(ax)
save_and_show(fig, "gen_mll.png")

In [ ]:
# ── Gen-level a and Z decay products ───────────────────────────────────────────
from collections import Counter

pdg_labels = {
    11: r"$e^-$", -11: r"$e^+$", 13: r"$\mu^-$", -13: r"$\mu^+$",
    15: r"$\tau^-$", -15: r"$\tau^+$", 211: r"$\pi^+$", -211: r"$\pi^-$",
    111: r"$\pi^0$", 22: r"$\gamma$", 321: r"$K^+$", -321: r"$K^-$",
    310: r"$K_S^0$", 130: r"$K_L^0$",
}

def get_decay_modes(events, parent_pdgId):
    """Get decay daughter PDG IDs for a given parent particle."""
    modes = Counter()
    daughter_pdgIds = []
    for ievt in range(len(events)):
        pdg = events.GenPart_pdgId[ievt]
        mom = events.GenPart_genPartIdxMother[ievt]
        flags = events.GenPart_statusFlags[ievt]
        for i in range(len(pdg)):
            if abs(pdg[i]) == parent_pdgId and bool(flags[i] & (1 << 13)):
                daughters = tuple(sorted([int(pdg[j]) for j in range(len(pdg)) if mom[j] == i]))
                modes[daughters] += 1
                for d in daughters:
                    daughter_pdgIds.append(abs(d))
    return modes, daughter_pdgIds

a_modes, a_daughter_ids = get_decay_modes(events, 36)
z_modes, z_daughter_ids = get_decay_modes(events, 23)

# Print decay tables
print("Gen-level a decay modes:")
for mode, count in a_modes.most_common():
    names = ", ".join(pdg_labels.get(p, str(p)) for p in mode)
    print(f"  a → {names}: {count} events ({100*count/sum(a_modes.values()):.0f}%)")

print("\nGen-level Z decay modes:")
for mode, count in z_modes.most_common():
    names = ", ".join(pdg_labels.get(p, str(p)) for p in mode)
    print(f"  Z → {names}: {count} events ({100*count/sum(z_modes.values()):.0f}%)")

# ── Plot: a decay daughter PDG IDs ────────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# a daughters bar chart by decay mode
a_mode_labels = []
a_mode_counts = []
for mode, count in a_modes.most_common():
    label = " + ".join(pdg_labels.get(p, str(p)) for p in mode)
    a_mode_labels.append(label)
    a_mode_counts.append(count)

axes[0].barh(range(len(a_mode_labels)), a_mode_counts, color="#2ecc71", edgecolor="#27ae60")
axes[0].set_yticks(range(len(a_mode_labels)))
axes[0].set_yticklabels(a_mode_labels, fontsize=14)
axes[0].set_xlabel("Events", fontsize=16)
axes[0].set_title(r"$a$ decay modes (gen-level)", fontsize=18)
axes[0].invert_yaxis()
for i, v in enumerate(a_mode_counts):
    axes[0].text(v + 0.5, i, str(v), va="center", fontsize=14)

# Z daughters bar chart by decay mode
z_mode_labels = []
z_mode_counts = []
for mode, count in z_modes.most_common():
    label = " + ".join(pdg_labels.get(p, str(p)) for p in mode)
    z_mode_labels.append(label)
    z_mode_counts.append(count)

axes[1].barh(range(len(z_mode_labels)), z_mode_counts, color="#3498db", edgecolor="#2980b9")
axes[1].set_yticks(range(len(z_mode_labels)))
axes[1].set_yticklabels(z_mode_labels, fontsize=14)
axes[1].set_xlabel("Events", fontsize=16)
axes[1].set_title(r"$Z$ decay modes (gen-level)", fontsize=18)
axes[1].invert_yaxis()
for i, v in enumerate(z_mode_counts):
    axes[1].text(v + 0.5, i, str(v), va="center", fontsize=14)

save_and_show(fig, "gen_decay_modes.png")

## Reconstructed Object Selection

Apply baseline selection cuts for muons, electrons, and jets following standard CMS Run 3 recommendations.

In [ ]:
# ── Muon selection ─────────────────────────────────────────────────────────────
mu_pt_cut  = events.Muon_pt > 5.0          # pT > 5 GeV
mu_eta_cut = np.abs(events.Muon_eta) < 2.4  # |η| < 2.4
mu_id_cut  = events.Muon_mediumId           # Medium ID
mu_iso_cut = events.Muon_pfRelIso04_all < 0.25  # PF relative isolation

mu_sel = mu_pt_cut & mu_eta_cut & mu_id_cut & mu_iso_cut

# Build muon 4-vectors
sel_mu = vector.zip({
    "pt":   events.Muon_pt[mu_sel],
    "eta":  events.Muon_eta[mu_sel],
    "phi":  events.Muon_phi[mu_sel],
    "mass": events.Muon_mass[mu_sel],
})
sel_mu_charge = events.Muon_charge[mu_sel]

# ── Electron selection ────────────────────────────────────────────────────────
el_pt_cut  = events.Electron_pt > 7.0           # pT > 7 GeV
el_eta_cut = np.abs(events.Electron_eta) < 2.5  # |η| < 2.5
el_id_cut  = events.Electron_cutBased >= 2       # Medium cut-based ID
el_iso_cut = events.Electron_pfRelIso03_all < 0.25

el_sel = el_pt_cut & el_eta_cut & el_id_cut & el_iso_cut

sel_el = vector.zip({
    "pt":   events.Electron_pt[el_sel],
    "eta":  events.Electron_eta[el_sel],
    "phi":  events.Electron_phi[el_sel],
    "mass": events.Electron_mass[el_sel],
})
sel_el_charge = events.Electron_charge[el_sel]

# ── Jet selection ─────────────────────────────────────────────────────────────
jet_pt_cut  = events.Jet_pt > 25.0           # pT > 25 GeV
jet_eta_cut = np.abs(events.Jet_eta) < 2.5   # |η| < 2.5

jet_sel = jet_pt_cut & jet_eta_cut

sel_jets = vector.zip({
    "pt":   events.Jet_pt[jet_sel],
    "eta":  events.Jet_eta[jet_sel],
    "phi":  events.Jet_phi[jet_sel],
    "mass": events.Jet_mass[jet_sel],
})
sel_jet_btag = events.Jet_btagDeepFlavB[jet_sel]

# Summary
n_mu  = ak.num(sel_mu)
n_el  = ak.num(sel_el)
n_jet = ak.num(sel_jets)

print(f"Selected muons:     mean {ak.mean(n_mu):.2f}/event,  total {ak.sum(n_mu)}")
print(f"Selected electrons: mean {ak.mean(n_el):.2f}/event,  total {ak.sum(n_el)}")
print(f"Selected jets:      mean {ak.mean(n_jet):.2f}/event, total {ak.sum(n_jet)}")

In [ ]:
# ── Reco lepton kinematic plots ────────────────────────────────────────────────
fig, axes = plt.subplots(2, 3, figsize=(21, 13))

# Muon pT
all_mu_pt = ak.flatten(sel_mu.pt).to_numpy()
h = hist.Hist(hist.axis.Regular(30, 0, 100, label=r"Muon $p_T$ [GeV]"))
h.fill(all_mu_pt)
hep.histplot(h, ax=axes[0, 0], histtype="fill", alpha=0.5, color="#3498db", edgecolor="#3498db")
axes[0, 0].set_ylabel("Muons / bin")
cms_label(axes[0, 0])

# Muon eta
all_mu_eta = ak.flatten(sel_mu.eta).to_numpy()
h = hist.Hist(hist.axis.Regular(30, -2.5, 2.5, label=r"Muon $\eta$"))
h.fill(all_mu_eta)
hep.histplot(h, ax=axes[0, 1], histtype="fill", alpha=0.5, color="#3498db", edgecolor="#3498db")
axes[0, 1].set_ylabel("Muons / bin")
cms_label(axes[0, 1])

# Muon multiplicity
h = hist.Hist(hist.axis.Regular(8, -0.5, 7.5, label=r"$N_{\mu}$ / event"))
h.fill(ak.to_numpy(n_mu))
hep.histplot(h, ax=axes[0, 2], histtype="fill", alpha=0.5, color="#3498db", edgecolor="#3498db")
axes[0, 2].set_ylabel("Events")
cms_label(axes[0, 2])

# Electron pT
all_el_pt = ak.flatten(sel_el.pt).to_numpy()
h = hist.Hist(hist.axis.Regular(30, 0, 100, label=r"Electron $p_T$ [GeV]"))
h.fill(all_el_pt)
hep.histplot(h, ax=axes[1, 0], histtype="fill", alpha=0.5, color="#e74c3c", edgecolor="#e74c3c")
axes[1, 0].set_ylabel("Electrons / bin")
cms_label(axes[1, 0])

# Electron eta
all_el_eta = ak.flatten(sel_el.eta).to_numpy()
h = hist.Hist(hist.axis.Regular(30, -2.5, 2.5, label=r"Electron $\eta$"))
h.fill(all_el_eta)
hep.histplot(h, ax=axes[1, 1], histtype="fill", alpha=0.5, color="#e74c3c", edgecolor="#e74c3c")
axes[1, 1].set_ylabel("Electrons / bin")
cms_label(axes[1, 1])

# Electron multiplicity
h = hist.Hist(hist.axis.Regular(8, -0.5, 7.5, label=r"$N_{e}$ / event"))
h.fill(ak.to_numpy(n_el))
hep.histplot(h, ax=axes[1, 2], histtype="fill", alpha=0.5, color="#e74c3c", edgecolor="#e74c3c")
axes[1, 2].set_ylabel("Events")
cms_label(axes[1, 2])

save_and_show(fig, "reco_lepton_kinematics.png")

## Z Boson Reconstruction

Reconstruct Z → ℓ⁺ℓ⁻ candidates from opposite-sign same-flavour (OSSF) lepton pairs. For events with multiple candidates, select the pair whose invariant mass is closest to $m_Z = 91.2$ GeV.

In [ ]:
# ── Z → μ⁺μ⁻ reconstruction ───────────────────────────────────────────────────
def reconstruct_z(leptons, charges, mz=91.2):
    """
    Reconstruct Z → ℓℓ from OSSF pairs, selecting the pair closest to mZ.
    Returns per-event Z candidate 4-vector (None-padded for events without a candidate).
    """
    # Build all pairs of indices
    n_lep = ak.num(leptons)
    idx = ak.local_index(leptons)
    pairs = ak.argcombinations(leptons, 2, axis=1)
    i0, i1 = ak.unzip(pairs)

    lep0 = leptons[i0]
    lep1 = leptons[i1]
    q0   = charges[i0]
    q1   = charges[i1]

    # Opposite-sign requirement
    os_mask = (q0 * q1 < 0)
    lep0 = lep0[os_mask]
    lep1 = lep1[os_mask]

    # Compute dilepton invariant mass
    z_cand = lep0 + lep1
    z_mass = z_cand.mass

    # Select pair closest to mZ
    dm = np.abs(z_mass - mz)
    best_idx = ak.argmin(dm, axis=1, keepdims=True)

    z_best = z_cand[best_idx]
    return z_best

# Reconstruct Z from muons
z_from_mu = reconstruct_z(sel_mu, sel_mu_charge)

# Reconstruct Z from electrons
z_from_el = reconstruct_z(sel_el, sel_el_charge)

# Combine: prefer the channel with mass closer to mZ
# Flatten to compare (pad with None for events without candidate)
z_mu_mass = ak.fill_none(ak.firsts(z_from_mu.mass), -999.0)
z_el_mass = ak.fill_none(ak.firsts(z_from_el.mass), -999.0)

prefer_mu = np.abs(z_mu_mass - 91.2) <= np.abs(z_el_mass - 91.2)
# If neither channel has a candidate, skip
has_z_mu = z_mu_mass > 0
has_z_el = z_el_mass > 0
has_z    = has_z_mu | has_z_el

# Build final Z candidate arrays
z_pt   = ak.where(prefer_mu, ak.fill_none(ak.firsts(z_from_mu.pt), 0.0),   ak.fill_none(ak.firsts(z_from_el.pt), 0.0))
z_eta  = ak.where(prefer_mu, ak.fill_none(ak.firsts(z_from_mu.eta), 0.0),  ak.fill_none(ak.firsts(z_from_el.eta), 0.0))
z_phi  = ak.where(prefer_mu, ak.fill_none(ak.firsts(z_from_mu.phi), 0.0),  ak.fill_none(ak.firsts(z_from_el.phi), 0.0))
z_mass = ak.where(prefer_mu, z_mu_mass, z_el_mass)
z_channel = ak.where(prefer_mu & has_z_mu, 1, ak.where(has_z_el, 2, 0))  # 1=μμ, 2=ee, 0=none

n_z = ak.sum(has_z)
n_z_mm = ak.sum(z_channel == 1)
n_z_ee = ak.sum(z_channel == 2)
print(f"Z candidates found: {n_z} total ({n_z_mm} μμ, {n_z_ee} ee)")
print(f"Events without Z:   {ak.sum(~has_z)}")

In [ ]:
# ── Z candidate plots ─────────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(21, 6))

# Z mass (all channels)
z_mass_arr = ak.to_numpy(z_mass[has_z])
h = hist.Hist(hist.axis.Regular(40, 0, 150, label=r"$m_{\ell\ell}$ [GeV]"))
h.fill(z_mass_arr)
hep.histplot(h, ax=axes[0], histtype="fill", alpha=0.5, color="#3498db", edgecolor="#3498db",
             label="All channels")
axes[0].axvline(91.2, color="black", linestyle="--", linewidth=1.5, label=r"$m_Z$ = 91.2 GeV")
axes[0].set_ylabel("Events")
axes[0].legend(fontsize=14)
cms_label(axes[0])

# Z mass by channel
z_mass_mm = ak.to_numpy(z_mass[z_channel == 1])
z_mass_ee = ak.to_numpy(z_mass[z_channel == 2])
h_mm = hist.Hist(hist.axis.Regular(40, 0, 150, label=r"$m_{\ell\ell}$ [GeV]"))
h_ee = hist.Hist(hist.axis.Regular(40, 0, 150, label=r"$m_{\ell\ell}$ [GeV]"))
h_mm.fill(z_mass_mm)
h_ee.fill(z_mass_ee)
hep.histplot([h_mm, h_ee], ax=axes[1], histtype="fill", alpha=0.5,
             label=[r"$Z \to \mu\mu$", r"$Z \to ee$"],
             color=["#3498db", "#e74c3c"], edgecolor=["#3498db", "#e74c3c"], stack=True)
axes[1].axvline(91.2, color="black", linestyle="--", linewidth=1.5)
axes[1].set_ylabel("Events")
axes[1].legend(fontsize=14)
cms_label(axes[1])

# Z pT
z_pt_arr = ak.to_numpy(z_pt[has_z])
h = hist.Hist(hist.axis.Regular(30, 0, 250, label=r"$p_T^{Z}$ [GeV]"))
h.fill(z_pt_arr)
hep.histplot(h, ax=axes[2], histtype="fill", alpha=0.5, color="#9b59b6", edgecolor="#9b59b6")
axes[2].set_ylabel("Events")
cms_label(axes[2])

save_and_show(fig, "reco_z_candidate.png")

## Pseudoscalar $a$ Decay and Reconstruction

First we inspect the gen-level $a$ decay products, then attempt to reconstruct $a$ candidates from the `IsoTrack` collection in NanoAOD.

**Expected branching fractions** (2HDM+S, $m_a = 1.0$ GeV): $a \to gg$ (~88%), $a \to \mu^+\mu^-$ (~12%).

**Reconstruction strategy (IsoTracks):**
1. Select high-purity isolated tracks not overlapping with the Z lepton candidates ($\Delta R > 0.3$)
2. Assign pion mass to each track and combine all opposite-sign pairs
3. Select the pair closest to $m_a$ as the $a$ candidate
4. Combine with the Z candidate to reconstruct $H \to Za$

In [ ]:
# ── Reconstruct a → tracks using IsoTracks ─────────────────────────────────────
# Build IsoTrack 4-vectors with pion mass hypothesis
M_PION = 0.13957  # GeV

# IsoTrack quality selection
trk_pt   = events["IsoTrack_pt"]
trk_eta  = events["IsoTrack_eta"]
trk_phi  = events["IsoTrack_phi"]
trk_q    = events["IsoTrack_charge"]
trk_iso  = events["IsoTrack_pfRelIso03_all"]
trk_pv   = events["IsoTrack_fromPV"]
trk_hp   = events["IsoTrack_isHighPurityTrack"]
trk_dxy  = events["IsoTrack_dxy"]
trk_dz   = events["IsoTrack_dz"]
trk_pdgId = events["IsoTrack_pdgId"]

trk_sel = (
    (trk_pt > 0.5)
    & (np.abs(trk_eta) < 2.5)
    & (trk_pv >= 2)
    & (trk_hp == 1)
    & (np.abs(trk_dxy) < 0.1)
    & (np.abs(trk_dz) < 0.5)
)

# Build 4-vectors for selected tracks
sel_trk_pt  = trk_pt[trk_sel]
sel_trk_eta = trk_eta[trk_sel]
sel_trk_phi = trk_phi[trk_sel]
sel_trk_q   = trk_q[trk_sel]
sel_trk_pdgId = trk_pdgId[trk_sel]

sel_tracks = vector.zip({
    "pt":   sel_trk_pt,
    "eta":  sel_trk_eta,
    "phi":  sel_trk_phi,
    "mass": M_PION * ak.ones_like(sel_trk_pt),
})

n_trk = ak.num(sel_tracks)
print(f"IsoTrack selection: {int(ak.sum(n_trk))} tracks pass quality cuts")
print(f"Events with ≥2 tracks: {int(ak.sum(n_trk >= 2))}")
print(f"Events with ≥1 track:  {int(ak.sum(n_trk >= 1))}")
print(f"Events with 0 tracks:  {int(ak.sum(n_trk == 0))}")

# Lepton overlap removal: remove tracks within ΔR < 0.3 of selected muons or electrons
# Build lepton list for overlap removal
def remove_overlap(tracks, leptons, dr_cut=0.3):
    """Remove tracks overlapping with any lepton within dr_cut."""
    if ak.sum(ak.num(leptons)) == 0:
        return ak.ones_like(tracks.pt, dtype=bool)
    # Cross-product: for each track, check distance to all leptons
    trk_eta_bc, lep_eta_bc = ak.unzip(ak.cartesian([tracks.eta, leptons.eta], axis=1, nested=True))
    trk_phi_bc, lep_phi_bc = ak.unzip(ak.cartesian([tracks.phi, leptons.phi], axis=1, nested=True))
    deta = trk_eta_bc - lep_eta_bc
    dphi = np.mod(trk_phi_bc - lep_phi_bc + np.pi, 2 * np.pi) - np.pi
    dr = np.sqrt(deta**2 + dphi**2)
    min_dr = ak.min(dr, axis=-1)
    return min_dr > dr_cut

not_mu_overlap = remove_overlap(sel_tracks, sel_mu)
not_el_overlap = remove_overlap(sel_tracks, sel_el)
no_overlap = not_mu_overlap & not_el_overlap

clean_tracks = sel_tracks[no_overlap]
clean_q = sel_trk_q[no_overlap]
clean_pdgId = sel_trk_pdgId[no_overlap]
n_clean_trk = ak.num(clean_tracks)

print(f"\nAfter lepton overlap removal:")
print(f"  Tracks remaining: {int(ak.sum(n_clean_trk))}")
print(f"  Events with ≥2 tracks: {int(ak.sum(n_clean_trk >= 2))}")

# Form opposite-sign track pairs → a candidates
pairs = ak.argcombinations(clean_tracks, 2, axis=1)
if ak.sum(ak.num(pairs)) > 0:
    i0, i1 = ak.unzip(pairs)
    t0 = clean_tracks[i0]
    t1 = clean_tracks[i1]
    q0 = clean_q[i0]
    q1 = clean_q[i1]

    os_mask = (q0 * q1 < 0)
    t0_os = t0[os_mask]
    t1_os = t1[os_mask]
    trk_pair_sum = t0_os + t1_os
    trk_pair_mass = trk_pair_sum.mass

    # Best a candidate: OS pair closest to MASS_A
    dm_a = np.abs(trk_pair_mass - MASS_A)
    best_idx = ak.argmin(dm_a, axis=1, keepdims=True)
    a_best = trk_pair_sum[best_idx]

    has_a_cand = (ak.num(trk_pair_mass) > 0)
    a_cand_mass = ak.fill_none(ak.firsts(a_best.mass), -999)
    a_cand_pt   = ak.fill_none(ak.firsts(a_best.pt), -999)
    a_cand_eta  = ak.fill_none(ak.firsts(a_best.eta), -999)
    a_cand_phi  = ak.fill_none(ak.firsts(a_best.phi), -999)
else:
    has_a_cand = ak.zeros_like(n_trk, dtype=bool)
    a_cand_mass = -999 * ak.ones_like(n_trk, dtype=float)
    a_cand_pt   = -999 * ak.ones_like(n_trk, dtype=float)
    a_cand_eta  = -999 * ak.ones_like(n_trk, dtype=float)
    a_cand_phi  = -999 * ak.ones_like(n_trk, dtype=float)
    trk_pair_mass = ak.Array([[] for _ in range(len(events))])

has_a_reco = has_a_cand & (a_cand_mass > 0)
print(f"\na candidates from OS track pairs: {int(ak.sum(has_a_reco))}")
print(f"Events without a candidate: {int(ak.sum(~has_a_reco))}")
print(f"\nNote: IsoTrack collection is sparse — low reconstruction efficiency is expected.")

In [ ]:
# ── a → track pair candidate plots ─────────────────────────────────────────────
fig, axes = plt.subplots(2, 3, figsize=(21, 13))

a_reco_m_arr = ak.to_numpy(a_cand_mass[has_a_reco]) if int(ak.sum(has_a_reco)) > 0 else np.array([])
a_reco_pt_arr = ak.to_numpy(a_cand_pt[has_a_reco]) if int(ak.sum(has_a_reco)) > 0 else np.array([])

# All OS track pair invariant mass
all_trk_pair_m = ak.flatten(trk_pair_mass)
if len(all_trk_pair_m) > 0:
    all_trk_pair_m = all_trk_pair_m.to_numpy()
else:
    all_trk_pair_m = np.array([])

h = hist.Hist(hist.axis.Regular(50, 0, 5, label=r"$m_{\mathrm{trk\,pair}}$ [GeV]"))
if len(all_trk_pair_m) > 0:
    h.fill(all_trk_pair_m)
hep.histplot(h, ax=axes[0, 0], histtype="fill", alpha=0.5, color="#2ecc71", edgecolor="#2ecc71",
             label=f"All OS track pairs (N={len(all_trk_pair_m)})")
axes[0, 0].axvline(MASS_A, color="black", linestyle="--", linewidth=1.5,
                   label=f"$m_a$ = {MASS_A} GeV")
axes[0, 0].set_ylabel("Track pairs / bin")
axes[0, 0].legend(fontsize=13)
cms_label(axes[0, 0])

# Zoom near m_a
h = hist.Hist(hist.axis.Regular(40, 0, 3, label=r"$m_{\mathrm{trk\,pair}}$ [GeV]"))
if len(all_trk_pair_m) > 0:
    h.fill(all_trk_pair_m)
hep.histplot(h, ax=axes[0, 1], histtype="fill", alpha=0.5, color="#2ecc71", edgecolor="#2ecc71",
             label="All OS track pairs")
axes[0, 1].axvline(MASS_A, color="black", linestyle="--", linewidth=1.5,
                   label=f"$m_a$ = {MASS_A} GeV")
axes[0, 1].set_ylabel("Track pairs / bin")
axes[0, 1].legend(fontsize=13)
cms_label(axes[0, 1])

# Best a candidate mass per event
h = hist.Hist(hist.axis.Regular(40, 0, 3, label=r"Best $m_a^{\mathrm{reco}}$ [GeV]"))
if len(a_reco_m_arr) > 0:
    h.fill(a_reco_m_arr)
hep.histplot(h, ax=axes[0, 2], histtype="fill", alpha=0.5, color="#27ae60", edgecolor="#27ae60",
             label=f"Best a candidate (N={len(a_reco_m_arr)})")
axes[0, 2].axvline(MASS_A, color="black", linestyle="--", linewidth=1.5)
axes[0, 2].set_ylabel("Events")
axes[0, 2].legend(fontsize=13)
cms_label(axes[0, 2])

# a candidate pT
h = hist.Hist(hist.axis.Regular(30, 0, 100, label=r"$p_T^{a}$ candidate [GeV]"))
if len(a_reco_pt_arr) > 0:
    h.fill(a_reco_pt_arr)
hep.histplot(h, ax=axes[1, 0], histtype="fill", alpha=0.5, color="#2ecc71", edgecolor="#2ecc71")
axes[1, 0].set_ylabel("Events")
cms_label(axes[1, 0])

# IsoTrack multiplicity (after cleaning)
h = hist.Hist(hist.axis.Regular(8, -0.5, 7.5, label=r"$N_{\mathrm{IsoTrack}}^{\mathrm{clean}}$ / event"))
h.fill(ak.to_numpy(n_clean_trk))
hep.histplot(h, ax=axes[1, 1], histtype="fill", alpha=0.5, color="#e67e22", edgecolor="#e67e22")
axes[1, 1].set_ylabel("Events")
axes[1, 1].axvline(1.5, color="red", linestyle=":", linewidth=1.5, alpha=0.7,
                   label=r"$\geq 2$ tracks needed for $a$ reco")
axes[1, 1].legend(fontsize=12)
cms_label(axes[1, 1])

# Gen vs reco a mass comparison
h_gen_a = hist.Hist(hist.axis.Regular(40, 0, 3, label=r"$m_a$ [GeV]"))
h_reco_a = hist.Hist(hist.axis.Regular(40, 0, 3, label=r"$m_a$ [GeV]"))
h_gen_a.fill(a_m)
if len(a_reco_m_arr) > 0:
    h_reco_a.fill(a_reco_m_arr)
hep.histplot(h_gen_a, ax=axes[1, 2], histtype="step", linewidth=2, color="#2ecc71", label="Gen $a$")
hep.histplot(h_reco_a, ax=axes[1, 2], histtype="step", linewidth=2, color="#e74c3c",
             label=r"Reco $a$ (tracks)")
axes[1, 2].axvline(MASS_A, color="black", linestyle="--", linewidth=1, alpha=0.5)
axes[1, 2].set_ylabel("Events / bin")
axes[1, 2].legend(fontsize=14)
cms_label(axes[1, 2])

save_and_show(fig, "reco_a_candidate.png")

In [ ]:
# ── Higgs → Za reconstruction ──────────────────────────────────────────────────
# Combine reco Z candidate (from ℓℓ) + reco a candidate (from IsoTracks)
has_both = has_z & has_a_reco

z_vec_for_h = vector.zip({
    "pt":   z_pt[has_both],
    "eta":  z_eta[has_both],
    "phi":  z_phi[has_both],
    "mass": z_mass[has_both],
})
a_vec_for_h = vector.zip({
    "pt":   a_cand_pt[has_both],
    "eta":  a_cand_eta[has_both],
    "phi":  a_cand_phi[has_both],
    "mass": a_cand_mass[has_both],
})

h_cand = z_vec_for_h + a_vec_for_h

print(f"Events with both Z and a candidates: {int(ak.sum(has_both))}")

fig, axes = plt.subplots(1, 3, figsize=(21, 6))

if int(ak.sum(has_both)) > 0:
    h_cand_m = ak.to_numpy(h_cand.mass)
    h_cand_pt = ak.to_numpy(h_cand.pt)
    dr_za_reco = ak.to_numpy(z_vec_for_h.deltaR(a_vec_for_h))
else:
    h_cand_m = np.array([])
    h_cand_pt = np.array([])
    dr_za_reco = np.array([])

# H candidate mass
h = hist.Hist(hist.axis.Regular(40, 0, 250, label=r"$m_{Za}$ [GeV]"))
if len(h_cand_m) > 0:
    h.fill(h_cand_m)
hep.histplot(h, ax=axes[0], histtype="fill", alpha=0.5, color="#e74c3c", edgecolor="#e74c3c",
             label=r"Reco $Z(\ell\ell) + a(\mathrm{tracks})$")
axes[0].axvline(125.0, color="black", linestyle="--", linewidth=1.5, label=r"$m_H$ = 125 GeV")
axes[0].set_ylabel("Events")
axes[0].legend(fontsize=14)
cms_label(axes[0])

# H candidate pT
h = hist.Hist(hist.axis.Regular(30, 0, 300, label=r"$p_T^{Za}$ [GeV]"))
if len(h_cand_pt) > 0:
    h.fill(h_cand_pt)
hep.histplot(h, ax=axes[1], histtype="fill", alpha=0.5, color="#e74c3c", edgecolor="#e74c3c")
axes[1].set_ylabel("Events")
cms_label(axes[1])

# ΔR(Z, a) reco
h = hist.Hist(hist.axis.Regular(25, 0, 5, label=r"$\Delta R(Z, a)$ reco"))
if len(dr_za_reco) > 0:
    h.fill(dr_za_reco)
hep.histplot(h, ax=axes[2], histtype="fill", alpha=0.5, color="#9b59b6", edgecolor="#9b59b6")
axes[2].set_ylabel("Events")
cms_label(axes[2])

save_and_show(fig, "reco_higgs_candidate.png")

## Pseudoscalar $a$ Reconstruction from PF Candidates

The `IsoTrack` collection in NanoAOD is sparse by design (only isolated tracks are stored).
For a low-mass pseudoscalar $a \to gg / q\bar{q}$, the decay products are soft and may
not pass isolation requirements.

Here we try the **PFCands** (particle flow candidates) collection, which stores *all*
charged PF candidates from the primary vertex. This gives much higher track multiplicity
and better sensitivity to soft, collimated $a$ decays.

**Strategy:**
1. Select charged PF candidates with $p_T > 0.5$ GeV, $|\eta| < 2.5$, associated to the PV
2. Remove candidates overlapping with the Z lepton candidates ($\Delta R > 0.3$)
3. Form opposite-sign pairs and select the one closest to $m_a$
4. Compare reconstruction efficiency with the IsoTrack approach

In [ ]:
# ── Reconstruct a → tracks using PF Candidates ─────────────────────────────────
if not HAS_PFCANDS:
    print("⚠ PFCands collection not found in this NanoAOD file.")
    print("  PFCands are available if NanoAOD was produced with the 'addPFCands' customisation,")
    print("  or with NanoAODv15 NANO step using --customise PhysicsTools/NanoAOD/custom_jme_cff.PrepJMECustomNanoAOD")
    print("  Skipping PFCands-based reconstruction.")
    has_a_pf = ak.zeros_like(events["nGenPart"], dtype=bool)
    a_pf_mass = -999 * ak.ones_like(events["nGenPart"], dtype=float)
    a_pf_pt   = -999 * ak.ones_like(events["nGenPart"], dtype=float)
    a_pf_eta  = -999 * ak.ones_like(events["nGenPart"], dtype=float)
    a_pf_phi  = -999 * ak.ones_like(events["nGenPart"], dtype=float)
    pf_pair_mass = ak.Array([[] for _ in range(len(events))])
else:
    print("✓ PFCands collection found — building PF candidate 4-vectors\n")

    # Load PFCands kinematics
    pf_pt  = events["PFCands_pt"]
    pf_eta = events["PFCands_eta"]
    pf_phi = events["PFCands_phi"]
    pf_mass = events["PFCands_mass"]
    pf_charge = events["PFCands_charge"]
    pf_pdgId = events["PFCands_pdgId"] if "PFCands_pdgId" in events.fields else None

    # PV association quality (if available)
    if "PFCands_fromPV" in events.fields:
        pf_pv = events["PFCands_fromPV"]
    elif "PFCands_pvAssocQuality" in events.fields:
        pf_pv = events["PFCands_pvAssocQuality"]
    else:
        pf_pv = ak.ones_like(pf_pt, dtype=int) * 3  # assume all from PV

    # d0 / dz if available
    pf_dz = events["PFCands_dz"] if "PFCands_dz" in events.fields else None
    pf_d0 = events["PFCands_d0"] if "PFCands_d0" in events.fields else None

    # Quality selection: charged PF candidates from primary vertex
    pf_sel = (
        (pf_charge != 0)             # charged only
        & (pf_pt > 0.5)             # pT > 0.5 GeV
        & (np.abs(pf_eta) < 2.5)    # within tracker acceptance
        & (pf_pv >= 2)              # associated to PV (PV tight or better)
    )
    if pf_dz is not None:
        pf_sel = pf_sel & (np.abs(pf_dz) < 0.5)
    if pf_d0 is not None:
        pf_sel = pf_sel & (np.abs(pf_d0) < 0.1)

    sel_pf_pt  = pf_pt[pf_sel]
    sel_pf_eta = pf_eta[pf_sel]
    sel_pf_phi = pf_phi[pf_sel]
    sel_pf_mass = pf_mass[pf_sel]
    sel_pf_q   = pf_charge[pf_sel]

    # Build 4-vectors
    pf_tracks = vector.zip({
        "pt":   sel_pf_pt,
        "eta":  sel_pf_eta,
        "phi":  sel_pf_phi,
        "mass": sel_pf_mass,  # PFCands stores actual mass (pion/kaon/etc.)
    })

    n_pf = ak.num(pf_tracks)
    print(f"PFCands selection: {int(ak.sum(n_pf))} candidates pass quality cuts")
    print(f"  Mean per event:   {float(ak.mean(n_pf)):.1f}")
    print(f"  Events with ≥2:   {int(ak.sum(n_pf >= 2))}")
    print(f"  Events with 0:    {int(ak.sum(n_pf == 0))}")

    # Compare with IsoTrack multiplicity
    print(f"\n  cf. IsoTrack (clean): mean {float(ak.mean(n_clean_trk)):.1f}, "
          f"≥2 in {int(ak.sum(n_clean_trk >= 2))} events")

    # Lepton overlap removal (same function as before)
    not_mu_overlap_pf = remove_overlap(pf_tracks, sel_mu, dr_cut=0.3)
    not_el_overlap_pf = remove_overlap(pf_tracks, sel_el, dr_cut=0.3)
    no_overlap_pf = not_mu_overlap_pf & not_el_overlap_pf

    clean_pf = pf_tracks[no_overlap_pf]
    clean_pf_q = sel_pf_q[no_overlap_pf]
    n_clean_pf = ak.num(clean_pf)

    print(f"\nAfter lepton overlap removal:")
    print(f"  PFCands remaining: {int(ak.sum(n_clean_pf))}")
    print(f"  Events with ≥2:   {int(ak.sum(n_clean_pf >= 2))}")

    # Form opposite-sign pairs → a candidates
    pf_pairs = ak.argcombinations(clean_pf, 2, axis=1)
    if ak.sum(ak.num(pf_pairs)) > 0:
        pi0, pi1 = ak.unzip(pf_pairs)
        pf0 = clean_pf[pi0]
        pf1 = clean_pf[pi1]
        q0_pf = clean_pf_q[pi0]
        q1_pf = clean_pf_q[pi1]

        os_mask_pf = (q0_pf * q1_pf < 0)
        pf0_os = pf0[os_mask_pf]
        pf1_os = pf1[os_mask_pf]
        pf_pair_sum = pf0_os + pf1_os
        pf_pair_mass = pf_pair_sum.mass

        # Best a candidate: OS pair closest to MASS_A
        dm_a_pf = np.abs(pf_pair_mass - MASS_A)
        best_idx_pf = ak.argmin(dm_a_pf, axis=1, keepdims=True)
        a_pf_best = pf_pair_sum[best_idx_pf]

        has_a_pf = (ak.num(pf_pair_mass) > 0)
        a_pf_mass = ak.fill_none(ak.firsts(a_pf_best.mass), -999)
        a_pf_pt   = ak.fill_none(ak.firsts(a_pf_best.pt), -999)
        a_pf_eta  = ak.fill_none(ak.firsts(a_pf_best.eta), -999)
        a_pf_phi  = ak.fill_none(ak.firsts(a_pf_best.phi), -999)
    else:
        has_a_pf = ak.zeros_like(n_pf, dtype=bool)
        a_pf_mass = -999 * ak.ones_like(n_pf, dtype=float)
        a_pf_pt   = -999 * ak.ones_like(n_pf, dtype=float)
        a_pf_eta  = -999 * ak.ones_like(n_pf, dtype=float)
        a_pf_phi  = -999 * ak.ones_like(n_pf, dtype=float)
        pf_pair_mass = ak.Array([[] for _ in range(len(events))])

    has_a_pf_reco = has_a_pf & (a_pf_mass > 0)
    print(f"\na candidates from PFCands OS pairs: {int(ak.sum(has_a_pf_reco))}")
    print(f"  cf. IsoTrack a candidates:         {int(ak.sum(has_a_reco))}")
    if int(ak.sum(has_a_reco)) > 0:
        improvement = int(ak.sum(has_a_pf_reco)) / int(ak.sum(has_a_reco))
        print(f"  PFCands / IsoTrack efficiency ratio: {improvement:.1f}x")

In [ ]:
# ── PFCands a candidate plots + comparison with IsoTrack ────────────────────────
fig, axes = plt.subplots(2, 3, figsize=(21, 13))

# --- Row 0: PFCands a candidate distributions ---

# All OS PF pair invariant mass
all_pf_pair_m = ak.flatten(pf_pair_mass) if HAS_PFCANDS else ak.Array([])
if len(all_pf_pair_m) > 0:
    all_pf_pair_m_np = ak.to_numpy(all_pf_pair_m)
else:
    all_pf_pair_m_np = np.array([])

h = hist.Hist(hist.axis.Regular(50, 0, 5, label=r"$m_{\mathrm{PF\,pair}}$ [GeV]"))
if len(all_pf_pair_m_np) > 0:
    h.fill(all_pf_pair_m_np)
hep.histplot(h, ax=axes[0, 0], histtype="fill", alpha=0.5, color="#3498db", edgecolor="#3498db",
             label=f"All OS PF pairs (N={len(all_pf_pair_m_np)})")
axes[0, 0].axvline(MASS_A, color="black", linestyle="--", linewidth=1.5,
                   label=f"$m_a$ = {MASS_A} GeV")
axes[0, 0].set_ylabel("PF pairs / bin")
axes[0, 0].legend(fontsize=13)
cms_label(axes[0, 0])

# Zoom near m_a
h = hist.Hist(hist.axis.Regular(40, 0, 3, label=r"$m_{\mathrm{PF\,pair}}$ [GeV]"))
if len(all_pf_pair_m_np) > 0:
    h.fill(all_pf_pair_m_np)
hep.histplot(h, ax=axes[0, 1], histtype="fill", alpha=0.5, color="#3498db", edgecolor="#3498db",
             label="All OS PF pairs")
axes[0, 1].axvline(MASS_A, color="black", linestyle="--", linewidth=1.5,
                   label=f"$m_a$ = {MASS_A} GeV")
axes[0, 1].set_ylabel("PF pairs / bin")
axes[0, 1].legend(fontsize=13)
cms_label(axes[0, 1])

# Best PF a candidate mass per event
a_pf_m_arr = ak.to_numpy(a_pf_mass[has_a_pf_reco]) if HAS_PFCANDS and int(ak.sum(has_a_pf_reco)) > 0 else np.array([])
h = hist.Hist(hist.axis.Regular(40, 0, 3, label=r"Best $m_a^{\mathrm{PF\,reco}}$ [GeV]"))
if len(a_pf_m_arr) > 0:
    h.fill(a_pf_m_arr)
hep.histplot(h, ax=axes[0, 2], histtype="fill", alpha=0.5, color="#2980b9", edgecolor="#2980b9",
             label=f"Best PF $a$ cand. (N={len(a_pf_m_arr)})")
axes[0, 2].axvline(MASS_A, color="black", linestyle="--", linewidth=1.5)
axes[0, 2].set_ylabel("Events")
axes[0, 2].legend(fontsize=13)
cms_label(axes[0, 2])

# --- Row 1: Comparison IsoTrack vs PFCands ---

# Track multiplicity comparison
h_iso = hist.Hist(hist.axis.Regular(30, -0.5, 29.5, label=r"$N_{\mathrm{tracks}}^{\mathrm{clean}}$ / event"))
h_pf  = hist.Hist(hist.axis.Regular(30, -0.5, 29.5, label=r"$N_{\mathrm{tracks}}^{\mathrm{clean}}$ / event"))
h_iso.fill(ak.to_numpy(n_clean_trk))
if HAS_PFCANDS:
    h_pf.fill(ak.to_numpy(n_clean_pf))
hep.histplot(h_iso, ax=axes[1, 0], histtype="step", linewidth=2, color="#e67e22", label="IsoTrack")
hep.histplot(h_pf, ax=axes[1, 0], histtype="step", linewidth=2, color="#3498db", label="PFCands")
axes[1, 0].set_ylabel("Events")
axes[1, 0].legend(fontsize=14)
axes[1, 0].set_yscale("log")
cms_label(axes[1, 0])

# Gen vs reco a mass comparison: IsoTrack vs PFCands
h_gen  = hist.Hist(hist.axis.Regular(40, 0, 3, label=r"$m_a$ [GeV]"))
h_iso_a = hist.Hist(hist.axis.Regular(40, 0, 3, label=r"$m_a$ [GeV]"))
h_pf_a  = hist.Hist(hist.axis.Regular(40, 0, 3, label=r"$m_a$ [GeV]"))
h_gen.fill(a_m)
if len(a_reco_m_arr) > 0:
    h_iso_a.fill(a_reco_m_arr)
if len(a_pf_m_arr) > 0:
    h_pf_a.fill(a_pf_m_arr)
hep.histplot(h_gen, ax=axes[1, 1], histtype="step", linewidth=2, color="#2ecc71", label="Gen $a$")
hep.histplot(h_iso_a, ax=axes[1, 1], histtype="step", linewidth=2, color="#e67e22",
             label=f"Reco $a$ IsoTrack (N={len(a_reco_m_arr)})")
hep.histplot(h_pf_a, ax=axes[1, 1], histtype="step", linewidth=2, color="#3498db",
             label=f"Reco $a$ PFCands (N={len(a_pf_m_arr)})")
axes[1, 1].axvline(MASS_A, color="black", linestyle="--", linewidth=1, alpha=0.5)
axes[1, 1].set_ylabel("Events / bin")
axes[1, 1].legend(fontsize=12)
cms_label(axes[1, 1])

# PFCands a candidate pT
a_pf_pt_arr = ak.to_numpy(a_pf_pt[has_a_pf_reco]) if HAS_PFCANDS and int(ak.sum(has_a_pf_reco)) > 0 else np.array([])
h_iso_pt = hist.Hist(hist.axis.Regular(30, 0, 100, label=r"$p_T^{a}$ candidate [GeV]"))
h_pf_pt  = hist.Hist(hist.axis.Regular(30, 0, 100, label=r"$p_T^{a}$ candidate [GeV]"))
if len(a_reco_pt_arr) > 0:
    h_iso_pt.fill(a_reco_pt_arr)
if len(a_pf_pt_arr) > 0:
    h_pf_pt.fill(a_pf_pt_arr)
hep.histplot(h_iso_pt, ax=axes[1, 2], histtype="step", linewidth=2, color="#e67e22", label="IsoTrack")
hep.histplot(h_pf_pt, ax=axes[1, 2], histtype="step", linewidth=2, color="#3498db", label="PFCands")
axes[1, 2].set_ylabel("Events")
axes[1, 2].legend(fontsize=14)
cms_label(axes[1, 2])

save_and_show(fig, "reco_a_pfcands_comparison.png")

In [ ]:
# ── Higgs → Za reconstruction using PFCands a candidate ─────────────────────────
has_both_pf = has_z & has_a_pf_reco

print(f"H → Za reconstruction comparison:")
print(f"  IsoTrack: {int(ak.sum(has_both))} events with Z + a(IsoTrack)")
print(f"  PFCands:  {int(ak.sum(has_both_pf))} events with Z + a(PFCands)")

if int(ak.sum(has_both_pf)) > 0:
    z_vec_pf = vector.zip({
        "pt":   z_pt[has_both_pf],
        "eta":  z_eta[has_both_pf],
        "phi":  z_phi[has_both_pf],
        "mass": z_mass[has_both_pf],
    })
    a_vec_pf = vector.zip({
        "pt":   a_pf_pt[has_both_pf],
        "eta":  a_pf_eta[has_both_pf],
        "phi":  a_pf_phi[has_both_pf],
        "mass": a_pf_mass[has_both_pf],
    })
    h_cand_pf = z_vec_pf + a_vec_pf
    h_pf_m = ak.to_numpy(h_cand_pf.mass)
    h_pf_pt_arr = ak.to_numpy(h_cand_pf.pt)
else:
    h_pf_m = np.array([])
    h_pf_pt_arr = np.array([])

fig, axes = plt.subplots(1, 3, figsize=(21, 6))

# H candidate mass: IsoTrack vs PFCands
h_iso_h = hist.Hist(hist.axis.Regular(40, 0, 250, label=r"$m_{Za}$ [GeV]"))
h_pf_h  = hist.Hist(hist.axis.Regular(40, 0, 250, label=r"$m_{Za}$ [GeV]"))
if len(h_cand_m) > 0:
    h_iso_h.fill(h_cand_m)
if len(h_pf_m) > 0:
    h_pf_h.fill(h_pf_m)
hep.histplot(h_iso_h, ax=axes[0], histtype="step", linewidth=2, color="#e67e22",
             label=f"IsoTrack (N={len(h_cand_m)})")
hep.histplot(h_pf_h, ax=axes[0], histtype="step", linewidth=2, color="#3498db",
             label=f"PFCands (N={len(h_pf_m)})")
axes[0].axvline(125.0, color="black", linestyle="--", linewidth=1.5, label=r"$m_H$ = 125 GeV")
axes[0].set_ylabel("Events")
axes[0].legend(fontsize=13)
cms_label(axes[0])

# H candidate pT comparison
h_iso_hpt = hist.Hist(hist.axis.Regular(30, 0, 300, label=r"$p_T^{Za}$ [GeV]"))
h_pf_hpt  = hist.Hist(hist.axis.Regular(30, 0, 300, label=r"$p_T^{Za}$ [GeV]"))
if len(h_cand_pt) > 0:
    h_iso_hpt.fill(h_cand_pt)
if len(h_pf_pt_arr) > 0:
    h_pf_hpt.fill(h_pf_pt_arr)
hep.histplot(h_iso_hpt, ax=axes[1], histtype="step", linewidth=2, color="#e67e22", label="IsoTrack")
hep.histplot(h_pf_hpt, ax=axes[1], histtype="step", linewidth=2, color="#3498db", label="PFCands")
axes[1].set_ylabel("Events")
axes[1].legend(fontsize=14)
cms_label(axes[1])

# Efficiency summary text
eff_text = (
    f"Reconstruction efficiency (Z + a):\n"
    f"  IsoTrack: {int(ak.sum(has_both))}/{len(events)} "
    f"= {100*int(ak.sum(has_both))/len(events):.1f}%\n"
    f"  PFCands:  {int(ak.sum(has_both_pf))}/{len(events)} "
    f"= {100*int(ak.sum(has_both_pf))/len(events):.1f}%"
)
axes[2].text(0.1, 0.6, eff_text, transform=axes[2].transAxes, fontsize=16,
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
axes[2].set_title("Reconstruction Efficiency Summary", fontsize=16)
axes[2].axis("off")

save_and_show(fig, "reco_higgs_isotrack_vs_pfcands.png")

## Jet and MET Distributions

Basic jet and missing transverse energy distributions. Jets arise mainly from ISR/FSR activity.

In [ ]:
# ── Jet and MET plots ─────────────────────────────────────────────────────────
fig, axes = plt.subplots(2, 3, figsize=(21, 13))

# Jet multiplicity
h = hist.Hist(hist.axis.Regular(10, -0.5, 9.5, label=r"$N_{\mathrm{jets}}$ / event"))
h.fill(ak.to_numpy(n_jet))
hep.histplot(h, ax=axes[0, 0], histtype="fill", alpha=0.5, color="#2ecc71", edgecolor="#2ecc71")
axes[0, 0].set_ylabel("Events")
cms_label(axes[0, 0])

# Leading jet pT
lead_jet_pt = ak.fill_none(ak.firsts(sel_jets.pt), -1)
has_jet = lead_jet_pt > 0
h = hist.Hist(hist.axis.Regular(30, 0, 200, label=r"Leading jet $p_T$ [GeV]"))
h.fill(ak.to_numpy(lead_jet_pt[has_jet]))
hep.histplot(h, ax=axes[0, 1], histtype="fill", alpha=0.5, color="#2ecc71", edgecolor="#2ecc71")
axes[0, 1].set_ylabel("Events")
cms_label(axes[0, 1])

# Leading jet eta
lead_jet_eta = ak.fill_none(ak.firsts(sel_jets.eta), -99)
h = hist.Hist(hist.axis.Regular(30, -2.5, 2.5, label=r"Leading jet $\eta$"))
h.fill(ak.to_numpy(lead_jet_eta[has_jet]))
hep.histplot(h, ax=axes[0, 2], histtype="fill", alpha=0.5, color="#2ecc71", edgecolor="#2ecc71")
axes[0, 2].set_ylabel("Events")
cms_label(axes[0, 2])

# All jet pT
all_jet_pt = ak.flatten(sel_jets.pt).to_numpy()
h = hist.Hist(hist.axis.Regular(30, 0, 200, label=r"Jet $p_T$ [GeV]"))
h.fill(all_jet_pt)
hep.histplot(h, ax=axes[1, 0], histtype="fill", alpha=0.5, color="#27ae60", edgecolor="#27ae60")
axes[1, 0].set_ylabel("Jets / bin")
cms_label(axes[1, 0])

# b-tag score (DeepFlavour)
all_btag = ak.flatten(sel_jet_btag).to_numpy()
h = hist.Hist(hist.axis.Regular(30, 0, 1, label="DeepFlavB score"))
h.fill(all_btag)
hep.histplot(h, ax=axes[1, 1], histtype="fill", alpha=0.5, color="#e67e22", edgecolor="#e67e22")
axes[1, 1].set_ylabel("Jets / bin")
axes[1, 1].set_yscale("log")
cms_label(axes[1, 1])

# PuppiMET
met_pt = ak.to_numpy(events.PuppiMET_pt)
h = hist.Hist(hist.axis.Regular(30, 0, 150, label=r"$p_T^{\mathrm{miss}}$ (Puppi) [GeV]"))
h.fill(met_pt)
hep.histplot(h, ax=axes[1, 2], histtype="fill", alpha=0.5, color="#8e44ad", edgecolor="#8e44ad")
axes[1, 2].set_ylabel("Events")
cms_label(axes[1, 2])

save_and_show(fig, "reco_jets_met.png")

## Event-Level Observables

Scalar sum of jet $p_T$ ($H_T$), total visible transverse momentum, and combined event-level quantities.

In [ ]:
# ── Event-level observables ────────────────────────────────────────────────────
# HT = scalar sum of jet pT
ht = ak.sum(sel_jets.pt, axis=1)

# Total lepton multiplicity
n_lep_total = n_mu + n_el

fig, axes = plt.subplots(1, 3, figsize=(21, 6))

# HT
h = hist.Hist(hist.axis.Regular(30, 0, 400, label=r"$H_T$ [GeV]"))
h.fill(ak.to_numpy(ht))
hep.histplot(h, ax=axes[0], histtype="fill", alpha=0.5, color="#1abc9c", edgecolor="#1abc9c")
axes[0].set_ylabel("Events")
cms_label(axes[0])

# Total lepton multiplicity
h = hist.Hist(hist.axis.Regular(8, -0.5, 7.5, label=r"$N_{\ell}$ / event"))
h.fill(ak.to_numpy(n_lep_total))
hep.histplot(h, ax=axes[1], histtype="fill", alpha=0.5, color="#e74c3c", edgecolor="#e74c3c")
axes[1].set_ylabel("Events")
cms_label(axes[1])

# MET phi
met_phi = ak.to_numpy(events.PuppiMET_phi)
h = hist.Hist(hist.axis.Regular(30, -np.pi, np.pi, label=r"$\phi^{\mathrm{miss}}$"))
h.fill(met_phi)
hep.histplot(h, ax=axes[2], histtype="fill", alpha=0.5, color="#8e44ad", edgecolor="#8e44ad")
axes[2].set_ylabel("Events")
cms_label(axes[2])

save_and_show(fig, "event_level.png")

## Generator vs Reconstruction Comparison

Compare gen-level and reco-level Z boson mass and $p_T$ to assess reconstruction performance.

In [ ]:
# ── Gen vs Reco comparison ─────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Z mass: gen vs reco
h_gen = hist.Hist(hist.axis.Regular(40, 0, 150, label=r"$m_{\ell\ell}$ [GeV]"))
h_reco = hist.Hist(hist.axis.Regular(40, 0, 150, label=r"$m_{\ell\ell}$ [GeV]"))
h_gen.fill(z_m)
h_reco.fill(z_mass_arr)
hep.histplot(h_gen, ax=axes[0], histtype="step", linewidth=2, color="#2ecc71", label="Gen-level Z")
hep.histplot(h_reco, ax=axes[0], histtype="step", linewidth=2, color="#e74c3c", label="Reco Z candidate")
axes[0].axvline(91.2, color="black", linestyle="--", linewidth=1, alpha=0.5)
axes[0].set_ylabel("Events / bin")
axes[0].legend(fontsize=14)
cms_label(axes[0])

# Z pT: gen vs reco
h_gen_pt = hist.Hist(hist.axis.Regular(30, 0, 250, label=r"$p_T^{Z}$ [GeV]"))
h_reco_pt = hist.Hist(hist.axis.Regular(30, 0, 250, label=r"$p_T^{Z}$ [GeV]"))
h_gen_pt.fill(z_pt_gen := ak.flatten(gen_Z.pt).to_numpy())
h_reco_pt.fill(z_pt_arr)
hep.histplot(h_gen_pt, ax=axes[1], histtype="step", linewidth=2, color="#2ecc71", label="Gen-level Z")
hep.histplot(h_reco_pt, ax=axes[1], histtype="step", linewidth=2, color="#e74c3c", label="Reco Z candidate")
axes[1].set_ylabel("Events / bin")
axes[1].legend(fontsize=14)
cms_label(axes[1])

save_and_show(fig, "gen_vs_reco_z.png")